In [ ]:
import wandb
import os

api = wandb.Api()

entity = "jduque"   
project = "MeltingPot" 
run_id = "ir1x5pcr"  

run = api.run(f"{entity}/{project}/{run_id}")
target_step = 20000
os.makedirs(f"wandb_videos/{run_id}/", exist_ok=True)

files = run.files()

for file in files:
    if "media/video" in file.name:  # Filters for files in the media/video/ directory
        step = int((file.name).split("_")[-2])
        if step == target_step:
            file_orig_clean_name = file.name.split("/")[-1]  # Get only the file name
            weird_hash = file_orig_clean_name.split(".gif")[0].split("_")[-1]
            file_clean_name = file_orig_clean_name.replace('_'+weird_hash, "")
            print("file_orig_clean_name", file_orig_clean_name)
            download_path = f"wandb_videos/{run_id}/{file_clean_name}"
            print(f"Downloading {file_clean_name} to {download_path}")
            file.download(root=f"wandb_videos/{run_id}/", replace=True)

            # Move the file to the correct directory without recreating the hierarchy
            os.rename(f"wandb_videos/{run_id}/media/videos/eval/{file_orig_clean_name}", download_path)

            # Optionally remove the empty directories after the move
            os.removedirs(f"wandb_videos/{run_id}/media/videos/eval/")

In [ ]:
gif_file = f"wandb_videos/{run_id}/video_commons_harvest__open_0_{target_step}.gif"

import matplotlib.pyplot as plt
from PIL import Image

gif = Image.open(gif_file)

num_frames = gif.n_frames
print(f'The GIF has {num_frames} frames.')

def extract_frames(gif, frame_indices):
    frames = []
    for index in frame_indices:
        gif.seek(index)
        frames.append(gif.copy())
    return frames

frames_to_plot = [2, 4, 6]
selected_frames = extract_frames(gif, frames_to_plot)

fig, axes = plt.subplots(1, len(selected_frames), figsize=(15, 5))
for i, frame in enumerate(selected_frames):
    axes[i].imshow(frame)
    axes[i].axis('off')

plt.savefig(f"frames.pdf", format="pdf", bbox_inches='tight', dpi=300)
plt.show()